<a href="https://colab.research.google.com/github/Mikael-lab/Proyecto-final-codigo-facilito/blob/main/Analisis_cuentas_bancarias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Exploración de datos de créditos Bancarios en su proceso de cobranza en el año 2010

Bienvenido, en este proyecto utiliza información obtenida de un proceso de cobranza.


*Cobranza*:
La cobranza es el acto o procedimiento por el cual se consigue la contraprestación por un bien o servicio o la cancelación de una deuda.

Referencia: https://economipedia.com/definiciones/cobranza.html


Para este proyecto nos enfocaremos en analizar la información de contacto con el deudor, dicho de otra manera, la gestión de la cuenta.


El objetivo final de cada gestión realizada a una cuenta es conseguir una promesa de pago por parte del titular.

Una promesa de pago es el registro de la fecha en que el titular se compromete a realizar el pago solicitado.

Dados estos escenarios, las preguntas que buscamos responder son las siguientes:

1. ¿Que cantidad de Gestiones se realizan en periodos de 1 mes?

2. ¿Que cantidad de promesas de pago se consiguieron en periodos de 1 mes?

3. ¿Hay relación entre el volumen de Gestiones y la cantidad de promesas de pago?


## Carga de datos originales

In [1]:
#Utilizaremos la libreria de pandas para manipular los datos originales
import pandas as pd


In [2]:
data = pd.read_csv('/content/drive/MyDrive/Proyecto final Data Science/gestiones_bnx_filtradas.csv')



In [3]:
# Damos un primer vistazo a nuestros datos
data

,IdGestion,IdCtaDesp,Fecha,Hora,IdTipoGestion,IdCodAccion,IdCodResultado,TipoPP,Monto,Comentarios
0,8081195,905361,2010-10-07,16:31:30,1,33,36,0.0,0.0,MENSAJE BUZON
1,8082138,915998,2010-10-07,18:34:57,1,33,42,0.0,0.0,240 NO REG TELEF DE TT
2,8082137,662872,2010-10-07,18:31:54,1,33,50,0.0,0.0,NO CONTESTA SIN OPCIONA BUZON
3,8082136,910303,2010-10-07,18:36:35,1,33,302,0.0,0.0,83595998 NUM FUERA DE SERVICIO
4,8082135,195590,2010-10-07,18:35:55,1,33,41,0.0,0.0,TEL DA TONO DE OCUPADO
...,...,...,...,...,...,...,...,...,...,...
1028384,8528500,230090,2010-12-27,19:47:00,3,38,27,0.0,0.0,SE DEJO NOTIFICACION CON TITULAR INFORMA PAGAR...
1028385,8528501,230090,2010-12-30,10:01:00,3,38,73,0.0,0.0,SE DEJO NOTIFICACION CON VECINO DE TITULAR DE ...
1028386,8528502,261142,2010-12-27,20:49:00,3,38,73,0.0,0.0,SE DEJO NOTIFICACION CON INQUILINO DE TITULAR ...
1028387,8528503,261142,2010-12-30,12:31:00,3,38,73,0.0,0.0,SE DEJO NOTIFICACION CON INQUILINO DE TITULAR ...


## Propiedades del conjunto de datos

In [4]:
# Para este punto vamos a utilizar la libreria funpymodelin
!pip install funpymodeling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 KB 3.4 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=54c438a774aa84b1d8675c796a25b0caadc7983d4607d760d6de5b143c643d31
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.2
    Uninstalling seaborn-0.11.2:
      Successfully uninstalled seaborn-0.11.2


In [5]:
from funpymodeling.exploratory import status

In [6]:
# q_nan = cantidad de nulos
# p_nan = porcentaje que representan los valores nulos
# q_zeros = cantidad de ceros
# p_zeros = porcentaje que representan los valores 0
# unique = cantidad de valores unicos
# type, object = string
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,IdGestion,0,0.000000,0,0.000000,1028389,int64
1,IdCtaDesp,0,0.000000,0,0.000000,102032,int64
2,Fecha,0,0.000000,0,0.000000,174,object
3,Hora,27882,0.027112,0,0.000000,62117,object
4,IdTipoGestion,0,0.000000,0,0.000000,9,int64
5,IdCodAccion,0,0.000000,0,0.000000,23,int64
6,IdCodResultado,0,0.000000,0,0.000000,156,int64
7,TipoPP,208857,0.203091,819532,0.796909,1,float64
8,Monto,207547,0.201818,761533,0.740511,10288,float64
9,Comentarios,1533,0.001491,0,0.000000,442211,object


Nuestro archivo de datos original contiene un tota de 10 columnas, para responder las interrogantes definidas en el proyecto solamente utilizaremos IdCodResultado, Fecha.

Observando la tabla de propiedades podemos saber que los campos que utilizaremos no tienen registros con valores nulos, esta se consiguio desde las reglas definidas en el proceso de cobranza:
- Toda gestión debe tener un resultado.
- Toda gestión debe tener una fecha en que se realizo.

In [7]:
# Creamos un nuevo data frame que contenga únicamente las 2 columnas que utilizaremos
df_fecha_codResultado = data[['Fecha','IdCodResultado']]

In [8]:
df_fecha_codResultado

,Fecha,IdCodResultado
0,2010-10-07,36
1,2010-10-07,42
2,2010-10-07,50
3,2010-10-07,302
4,2010-10-07,41
...,...,...
1028384,2010-12-27,27
1028385,2010-12-30,73
1028386,2010-12-27,73
1028387,2010-12-30,73


In [31]:
# Para poder gráficas nuestro dataFrame vamos a convertir los campos
# Fecha lo convertimos a DateTime
# IdCodResultado lo convertimos a String

df_fecha_codResultado.Fecha = pd.to_datetime(df_fecha_codResultado.Fecha,dayfirst=True)
df_fecha_codResultado.IdCodResultado = df_fecha_codResultado['IdCodResultado'].astype('string')

# Realizamos un crosstab y lo almacenamos en un nuevo dataFrame
df_fecha_coResultado_group = pd.crosstab(df_fecha_codResultado.Fecha, df_fecha_codResultado.IdCodResultado).resample("M").sum().reset_index()

# Creamos una nueva columna donde sumamos los valores de la columna 49 y 82 que el sistema de informacion tiene identificado como promesa de pago
df_fecha_coResultado_group['sum_promesas'] = df_fecha_coResultado_group['49'] + df_fecha_coResultado_group['82']

df_fecha_coResultado_group


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


IdCodResultado,Fecha,100,103,105,106,107,108,109,110,111,...,90,91,92,93,94,95,96,98,99,sum_promesas
0,2010-07-31,8,0,0,0,0,0,1,3,27,...,37,0,0,0,45,0,1,0,1,55
1,2010-08-31,1772,1,44,1,0,1,121,2,976,...,2334,0,26,996,5158,53,25,6,51,5777
2,2010-09-30,864,0,29,2,0,0,59,0,661,...,1586,37,12,388,3911,48,15,2,46,4372
3,2010-10-31,543,0,91,30,2,28,142,0,839,...,1506,0,16,39,3270,47,2,2,7,4439
4,2010-11-30,984,0,155,94,4,20,181,0,872,...,1644,0,15,15,3550,46,6,3,15,4133
5,2010-12-31,1252,0,236,151,2,59,110,0,582,...,1465,0,10,14,3032,58,5,3,20,3772


# Visualización de datos

In [32]:
# Para la visualización de datos vamos a utilizar la librería Altair
import altair as alt

In [33]:
# Promesas de pago por mes
c1 = alt.Chart(df_fecha_coResultado_group).mark_bar().encode(
    y= alt.Y("sum_promesas", title="Promesas"),
    x= alt.X("Fecha", title = "",timeUnit='month'),
    color = alt.Color('sum_promesas', title = "Volumen", scale = alt.Scale( scheme = "blues")),
).properties(
    title="Promesas de pago en el segundo semestre del año 2010"
)
    
text_line = c1.mark_text(
    dx=35,
    dy=-15
).encode( 
    text=alt.Text('sum_promesas',format=",.0f") 
)

(c1+text_line).properties(
    width=490,
    height=330
).configure_title(
    fontSize=15,
    dy= -15
)

alt.LayerChart(...)

Con el trabajo que hemos realizado hasta este punto ya pudimos identificar **el mes en el que se registraron más promesas de pago, que fue Agosto con un total de 5,777**.

El paso siguiente es saber el volumen de gestiones totales en comparacion con la cantidad de promesas de pago

In [37]:
# Obtenemos los totales por mes de todos los codigos de resultado
# con la función crosstab hacemos el cruce para tener como columnas los IdCodResultado
# la función resample nos ayuda a poner como fila el mes
# la función sum para hacer la sumatoria de las veces que aparece un IdCodResultado

df_totales = pd.crosstab(df_fecha_codResultado.Fecha,df_fecha_codResultado.IdCodResultado).resample("M").sum().reset_index()

# Creamos la columna y fila del total de IdCodResultado que sería equivalente a contar las gestiones
df_totales.loc['total', :] = df_totales.sum(numeric_only=True)
df_totales['total'] = df_totales.sum(axis=1, numeric_only=True)

# Creamos una nueva columna donde sumamos los valores de la columna 49 y 82 que el sistema de informacion tiene identificado como promesa de pago
df_totales['sum_promesas'] = df_totales['49'] + df_totales['82']

df_totales


IdCodResultado,Fecha,100,103,105,106,107,108,109,110,111,...,91,92,93,94,95,96,98,99,total,sum_promesas
0,2010-07-31,8.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,27.0,...,0.0,0.0,0.0,45.0,0.0,1.0,0.0,1.0,11552.0,55.0
1,2010-08-31,1772.0,1.0,44.0,1.0,0.0,1.0,121.0,2.0,976.0,...,0.0,26.0,996.0,5158.0,53.0,25.0,6.0,51.0,321256.0,5777.0
2,2010-09-30,864.0,0.0,29.0,2.0,0.0,0.0,59.0,0.0,661.0,...,37.0,12.0,388.0,3911.0,48.0,15.0,2.0,46.0,217595.0,4372.0
3,2010-10-31,543.0,0.0,91.0,30.0,2.0,28.0,142.0,0.0,839.0,...,0.0,16.0,39.0,3270.0,47.0,2.0,2.0,7.0,192522.0,4439.0
4,2010-11-30,984.0,0.0,155.0,94.0,4.0,20.0,181.0,0.0,872.0,...,0.0,15.0,15.0,3550.0,46.0,6.0,3.0,15.0,146826.0,4133.0
5,2010-12-31,1252.0,0.0,236.0,151.0,2.0,59.0,110.0,0.0,582.0,...,0.0,10.0,14.0,3032.0,58.0,5.0,3.0,20.0,138638.0,3772.0
total,NaT,5423.0,1.0,555.0,278.0,8.0,108.0,614.0,5.0,3957.0,...,37.0,79.0,1452.0,18966.0,252.0,54.0,16.0,140.0,1028389.0,22548.0


In [38]:
# Creamos los objetos para graficar
scale = alt.Scale(domain=["Promesas","Gestiones"], range=['blue','gray'])

base = alt.Chart(df_totales).transform_calculate(
    line="'Promesas'",
    bar="'Gestiones'"
).encode(
    alt.X("Fecha",title="",timeUnit='month'),
).properties(
    title="Volumen de gestiones y cantidad de promesas conseguidas"
)

bar = base.mark_bar().encode(
    alt.Y('total',title="Gestiones"),
    tooltip=[alt.Tooltip("total", title='Gestiones')],
    color=alt.Color('bar:N', scale=scale, title=''),
)

line = base.mark_line(point=alt.OverlayMarkDef(color="red")).encode(
    alt.Y("sum_promesas",title=""),
    tooltip=[alt.Tooltip('sum_promesas',title='Promesas')],
    color=alt.Color('line:N', scale=scale, title='')
)

text_line = line.mark_text(
    align='right',
    baseline='bottom',
    dx=3
).encode( 
    text=alt.Text('sum_promesas',format=",.0f")
)

text_bar = bar.mark_text(
    dx=35,
    dy=-15
).encode( 
    text=alt.Text('total',format=",.0f") 
)

((bar+text_bar)+(line+text_line)).properties(
    width=490,
    height=330
).configure_title(
    fontSize=15,
    dy= -15
)


alt.LayerChart(...)

Con la siguiente gráfica podemos observar lo siguiente:

1. El volumen de gestiones es abismalmente superior a la cantidad de promesas de pago generadas en cada mes.

2. El volumen de gestiones no siempre es proporcional a la cantidad de promesas de pago obtenidas. 
En el mes de Septiembre se obtuvieron un total de 3,952 promesas de pago con un volumen de gestiones de 217,595, en el mes de Octubre se obtuvieron 4,296 con un volumen de 192,522.

# Conclusiones
En este punto ya podemos contestar las preguntas que planteamos al principio.

1. ¿Que cantidad de Gestiones se realizan en periodos de 1 mes?
2. ¿Que cantidad de promesas de pago se consiguieron en periodos de 1 mes?
**R= se puede visualizar en la gráfica "Volumen de gestiones y cantidad de promesas conseguidas"**

3. ¿Hay relación entre el volumen de Gestiones y la cantidad de promesas de pago? 
**R= con los datos que se tienen hasta el momento no se observa una relación que nos indique que a mayor volumen de gestiones mayor cantidad de promesas de pago se van a obtener, claro que esto es forma general.**

# Segunda parte

Con el análisis anterior, surgieron nuevas preguntas las cuales podrían esclarecer la manera en que se trata a una cuenta.

Las preguntas que surgieron fueron las siguientes:

1. ¿Que cantidad de gestiones recibe en promedio una cuenta?

1. ¿En periodos de 1 mes se llega a gestionar todas las cuentas?

In [39]:
# Creamos una copia del dataframe original
df_copy = data
# Convertimos el campo a datetime
df_copy.Fecha = pd.to_datetime(df_copy.Fecha,dayfirst=True)

# Convertimos a entero el la fecha y la insertamos en una nueva columna llamada Mes
df_copy.insert(0,'Mes', (data['Fecha'].dt.month).astype(int) )





ValueError: ignored

In [15]:
# Agrupamos primero por mes y despues por IdCtaDesp para no hacer un conteo doble de gestiones
df_gestiones_promedio = df_copy.groupby(['Mes','IdCtaDesp'],as_index=False).IdCtaDesp.count()

df_gestiones_promedio


,Mes,IdCtaDesp
0,7,1
1,7,1
2,7,2
3,7,1
4,7,1
...,...,...
188515,12,12
188516,12,7
188517,12,23
188518,12,35


In [40]:
# Agrupamos por el mes, agregando el promedio del campo IdCTaDesp que contiene el total de gestiones y los redondeamos
df_promedio_final = df_gestiones_promedio.groupby(['Mes'],as_index=False).mean('IdCtaDesp').round()

# Convertimos el mes a texto para mejor visualización al momento de generar el gráfico
df_promedio_final['Mes'] = pd.to_datetime(df_promedio_final['Mes'], format='%m').dt.strftime('%b')

df_promedio_final =df_promedio_final.sort_index()

df_promedio_final


,Mes,IdCtaDesp
0,Jul,2.0
1,Aug,5.0
2,Sep,5.0
3,Oct,6.0
4,Nov,7.0
5,Dec,7.0


In [44]:
# Promedio de gestiones realizadas a una cuenta por mes
 
bar = alt.Chart(df_promedio_final).mark_bar().encode(
    y= alt.Y("IdCtaDesp", title="Gestiones"),
    x= alt.X("Mes", title = "",sort=['Mes']),
    color = alt.Color('IdCtaDesp', title = "Gestiones", scale = alt.Scale( scheme = "blues")),
    tooltip=[alt.Tooltip('IdCtaDesp',title="Gestiones")]
).properties(
    title="Promedio de gestiones por cuenta",
    width = 450
)

text_c1 = bar.mark_text(
    dx=0,
    dy=-15
).encode( 
    text=alt.Text('IdCtaDesp',format=",.0f") 
)

c1 = bar+text_c1

line = base.mark_line(point=alt.OverlayMarkDef(color="red")).encode(
    alt.Y("sum_promesas",title=""),
    tooltip=[alt.Tooltip('sum_promesas',title='Promesas')],
    color=alt.Color('line:N', scale=scale, title='')
)

text_line = line.mark_text(
    align='right',
    baseline='bottom',
    dx=3
).encode( 
    text='sum_promesas'
)

c2 = line+text_line



(c1 | c2)




alt.HConcatChart(...)

Obteniendo el promedio de gestiones que recibe una cuenta en un mes, podemos observar que va en aumento, sin embargo las promesas obtenidas no son proporcionales, lo que pudiera indicarnos `que la insistencia en la cobranza no siempre es la clave para conseguir una promesa de pago`, esto considerando que la mayor parte de las cuentas recibiera una gestión, sin embargo, en la Gráfica 2 se observa que el volumen de gestiones disminuye.

In [42]:
# En el dataFrame df_gestiones_promedio ya teniamos agrupadas las cuentas, entonces se eliminaron los duplicados
# por lo cual vamos a hacer el conteo en vez del promedio para saber que cantidad de cuentas se gestionan en periodos de 1 mes
df_ctas_mes = df_gestiones_promedio.groupby(['Mes'],as_index=False).count()

df_ctas_mes['Mes'] = pd.to_datetime(df_ctas_mes['Mes'], format='%m').dt.strftime('%b')

df_ctas_mes


,Mes,IdCtaDesp
0,Jul,6076
1,Aug,62006
2,Sep,44814
3,Oct,33454
4,Nov,21672
5,Dec,20498


In [43]:
# importamos spacing de numpy para la separación de gráficas
from numpy import spacing

bar = alt.Chart(df_ctas_mes).mark_bar().encode(
    y= alt.Y("IdCtaDesp", title="Cuentas"),
    x= alt.X("Mes", title = "",sort=['Mes']),
    color = alt.Color('IdCtaDesp', title = "Cuentas", scale = alt.Scale( scheme = "blues")),
    tooltip=[alt.Tooltip('IdCtaDesp',title="Cuentas")]
).properties(
    title="Total de Cuentas Gestionadas por mes",
    width=500
)

text_c1 = bar.mark_text(
    dx=0,
    dy=-15
).encode( 
    text=alt.Text('IdCtaDesp',format=",.0f") 
)

c1 = bar + text_c1

bar = alt.Chart(df_totales).mark_bar().encode(
    alt.X("Fecha",title="",timeUnit='month'),
    alt.Y('total',title="Gestiones"),
    color = alt.Color('total', title = "Gestiones", scale = alt.Scale( scheme = "teals")),
).properties(
    title="Volumen de Gestiones por mes",
    
)

text_c2 = bar.mark_text(
    dx=35,
    dy=-15
).encode( 
    text=alt.Text('total',format=",.0f") 
)

c2 = bar + text_c2

combined = alt.hconcat(
    c1.properties(
        width=490,
        height=330), 
    c2.properties(
        width=490,
        height=330
    )
).resolve_scale(
    color='independent'
).properties(
    spacing=100
)

combined

alt.HConcatChart(...)

Con las gráficas anteriores podemos ver que la cantidad de cuentas gestionadas decrementa al igual que el volumen de gestiones en cada uno de los periodos, entonces empezamos a tener `menos cuentas gestionadas, menos volumen de gestiones y el promedio de gestiones por cuenta aumenta.`


# Conclusiones
Las preguntas que surgieron fueron las siguientes:

¿Que cantidad de gestiones recibe en promedio una cuenta?
**R= rango de 2 a 7 en el semestre**
¿En periodos de 1 mes se llega a gestionar todas las cuentas?
**R= Únicamente en el mes de Agosto se gestionaron todas las cuentas

Para complementar las respuestas a las interrogantes se deben mencionar escenarios que pasan en la operacion de la agencia de cobranza, que son los siguientes:


- **Disminución de cuentas a gestionar:** esta disminución de cuentas puede darse por que existen cuentas que su deuda puede quedar liquidada, al entrar en este estado ya no se contempla dentro del universo de cuentas a gestionar.

- **Cantidad de gestiones por cuenta:** existen cuentas en las cuales obtienes una promesa de pago a la primer gestion, hay cuentas que hasta la *n* gestion obtienes una promesa de pago, estas gestiones previas a una promesa de pago se quedan registradas para que el la persona que interactue con el titular de la cuenta sepa el proceso de gestión de la misma.

# Conclusión final


Para la conclusión final vamos a obtener la diferencia y el porcentaje de variación de uno de los puntos claves, las promesas de pago.


In [56]:
# Utilizamos el dataframe donde hicimos la sumatoria de las promesas
df_variacion = df_totales[['Fecha','sum_promesas']]

# utilizamos la funcion drop para eliminar la fila 0
# ya que la sumatoria de promesas no es significante en comparacion con los demas datos
df_variacion.drop([0], axis=0, inplace=True) 

df_variacion


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


IdCodResultado,Fecha,sum_promesas
1,2010-08-31,5777.0
2,2010-09-30,4372.0
3,2010-10-31,4439.0
4,2010-11-30,4133.0
5,2010-12-31,3772.0
total,NaT,22548.0


In [57]:
# Creamos la columna diferencia por cada periodo
df_variacion['diferencia'] = df_variacion.sum_promesas.diff()
df_variacion

<ipython-input-57-1af8a2cfbb8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_variacion['diferencia'] = df_variacion.sum_promesas.diff()


IdCodResultado,Fecha,sum_promesas,diferencia
1,2010-08-31,5777.0,NaN
2,2010-09-30,4372.0,-1405.0
3,2010-10-31,4439.0,67.0
4,2010-11-30,4133.0,-306.0
5,2010-12-31,3772.0,-361.0
total,NaT,22548.0,18776.0


In [58]:
# Calculamos la variacion dividiendo la diferencia entre la suma de promesas de la fila anterior usando la función shift(1)
df_variacion['variacion'] = df_variacion.diferencia/df_variacion.sum_promesas.shift(1)*100
df_variacion

<ipython-input-58-feeb16b8b8eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_variacion['variacion'] = df_variacion.diferencia/df_variacion.sum_promesas.shift(1)*100


IdCodResultado,Fecha,sum_promesas,diferencia,variacion
1,2010-08-31,5777.0,NaN,NaN
2,2010-09-30,4372.0,-1405.0,-24.320582
3,2010-10-31,4439.0,67.0,1.532479
4,2010-11-30,4133.0,-306.0,-6.893444
5,2010-12-31,3772.0,-361.0,-8.734575
total,NaT,22548.0,18776.0,497.773065


La tendencia en promesas de pagos es a la bajalo cual indicaría a simple vista que el proceso de cobranza no esta siendo efectivo, sin embargo hay que tomar en cuenta que la cantidad de cuentas gestionadas fue disminuyendo, lo cual nos generá más preguntas como: 

1. ¿Cuales son los criterios para que una cuenta se elimine del universo de cuentas a ser gestionadas?

2. El factor humano de la persona que realiza la campaña, ¿Que tanto unfluye? o si la empresa tiene un estandár de campañas de cobranza.

Entre más "forma" van tomando los datos, más preguntas empiezan a surgir, entre más preguntas se vayan contestando, mejores decisiones se podrán tomar en caso contrario podemos saber que areas de oportunidad se tienen y trabajar en ello.